In [44]:
import pandas as pd
import plotly.express as px
from scipy.stats import chi2_contingency
import numpy as np

In [2]:
df = pd.read_excel("data/Telco_customer_churn_adapted_v2.xlsx")
df.head()

,Customer ID,Tenure Months,Location,Device Class,Games Product,Music Product,Education Product,Call Center,Video Product,Use MyApp,Payment Method,Monthly Purchase (Thou. IDR),Churn Label,Longitude,Latitude,CLTV (Predicted Thou. IDR)
0,0,2,Jakarta,Mid End,Yes,Yes,No,No,No,No,Digital Wallet,70.005,Yes,106.816666,-6.2,4210.7
1,1,2,Jakarta,High End,No,No,No,No,No,No,Pulsa,91.910,Yes,106.816666,-6.2,3511.3
2,2,8,Jakarta,High End,No,No,Yes,No,Yes,Yes,Pulsa,129.545,Yes,106.816666,-6.2,6983.6
3,3,28,Jakarta,High End,No,No,Yes,Yes,Yes,Yes,Pulsa,136.240,Yes,106.816666,-6.2,6503.9
4,4,49,Jakarta,High End,No,Yes,Yes,No,Yes,Yes,Debit,134.810,Yes,106.816666,-6.2,6942.0


In [90]:
products = ["Games Product", "Music Product", "Education Product", "Video Product", "Call Center", "Use MyApp", "Device Class"]

In [ ]:
idx = 1 

# Menghitung "Count" untuk setiap kategori
temp = df[[products[idx], "Churn Label", "Customer ID"]].groupby([products[idx], "Churn Label"]).count().reset_index()
temp.columns = [products[idx], "Churn Label", "Count"]

# Menghitung total "Count" untuk setiap kategori produk
total_count = temp.groupby(products[idx])["Count"].sum()
temp["Percentage"] = (temp["Count"] / temp[products[idx]].map(total_count)) * 100

# Plot dengan sumbu y "Count"
fig_count = px.bar(temp, x=products[idx], y="Count", color="Churn Label", barmode="group",
                   title=f"Jumlah Customer Churn dan Tidak Churn berdasarkan {products[idx]}")
fig_count.show()

# Plot dengan sumbu y "Percentage"
fig_percentage = px.bar(temp, x=products[idx], y="Percentage", color="Churn Label", barmode="group",
                        title=f"Persentase Customer Churn dan Tidak Churn berdasarkan {products[idx]}")
fig_percentage.show()

observed = pd.crosstab(df[products[idx]], df["Churn Label"])
chi2, p, dof, expected = chi2_contingency(observed)

if p < 0.05:
    print(f"Terdapat hubungan yang signifikan antara {products[idx]} dan Churn.")
else:
    print(f"Tidak terdapat hubungan yang signifikan antara {products[idx]} dan Churn.")


In [82]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

def cramers_v(cross_tabs):
    """
    Prints the degrees of freedom, effect size thresholds, Cramer's V value, and interpretation.
    
    Args:
        cross_tabs: A crosstab dataframe.
    """
    
    # Effect size data frame for Cramer's V function
    data = np.array([[1, .1, .3, .5],
       [2, .07, .21, .35],
       [3, .06, .17, .29],
       [4, .05, .15, .25],
       [5, .04, .13, .22]])
    sizes = pd.DataFrame(data, columns=['Degrees of Freedom', 'Small Effect', 'Medium Effect', 'Large Effect']) 
    
    # Getting the chi-squared stat
    chi2 = chi2_contingency(cross_tabs)[0]
    # Calculating the total number of observations
    n = cross_tabs.sum().sum()
    # Getting the degrees of freedom
    dof = min(cross_tabs.shape) - 1
    # Calculating Cramer's V
    v = np.sqrt(chi2 / (n * dof))
    
    # Interpreting the effect size
    effect_size = "Small Effect"
    for i, row in sizes.iterrows():
        print(row)
        if dof == row["Degrees of Freedom"]:
            if v > row["Small Effect"] and v <= row["Medium Effect"]:
                effect_size = "Small Effect"
            elif v > row["Medium Effect"] and v <= row["Large Effect"]:
                effect_size = "Medium Effect"
            else:
                effect_size = "Large Effect"
            break

    # Printing results
    print(f'V = {v}')
    print(f'Cramer\'s V Degrees of Freedom = {dof}')
    print(f'Effect Size Interpretation: {effect_size}')
    # print(f'\nEffect Size Thresholds\n{sizes}\n')


In [ ]:
observed = pd.crosstab(df["Games Product"], df["Churn Label"])
cramers_v(observed)

In [ ]:
# Effect size data frame for Cramer's V function
data = np.array([[1, .1, .3, .5],
    [2, .07, .21, .35],
    [3, .06, .17, .29],
    [4, .05, .15, .25],
    [5, .04, .13, .22]])
sizes = pd.DataFrame(data, columns=['Degrees of Freedom', 'Small Effect', 'Medium Effect', 'Large Effect']) 
sizes

In [ ]:
for i in range(len(products)):
    print(products[i])
    observed = pd.crosstab(df[products[i]], df["Churn Label"])
    cramers_v(observed)
    print()

In [93]:
def products_figure(product, percentage, dependent="Churn Label"):
    # Menghitung "Count" untuk setiap kategori
    temp = df[[product, "Churn Label", "Customer ID"]].groupby([product, "Churn Label"]).count().reset_index()
    temp.columns = [product, "Churn Label", "Count"]

    # Menghitung total "Count" untuk setiap kategori produk
    total_count = temp.groupby(product)["Count"].sum()
    temp["Percentage"] = (temp["Count"] / temp[product].map(total_count)) * 100

    # Plot dengan sumbu y "Count"
    y = "Percentage" if percentage else "Count"
    fig = px.bar(temp, x=product, y=y, color="Churn Label", barmode="group",
                 title=f"Jumlah Customer Churn dan Tidak Churn berdasarkan {product}")

    return fig

In [104]:
def chi2_test(product):
    observed = pd.crosstab(df[product], df["Churn Label"])
    chi2, p, dof, expected = chi2_contingency(observed)

    if p < 0.05:
        conclusion = f"Terdapat hubungan yang signifikan antara {product} dan Churn."
    else:
        conclusion = f"Tidak terdapat hubungan yang signifikan antara {product} dan Churn." 

    output = pd.DataFrame({
        "chi2" : [round(chi2, 2)],
        "p-value" : [round(p, 3)],
        "degrees of freedom" : [dof],
        "conclusion" : [conclusion]
    })

    return output

In [110]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

def cramers_v(product):
    cross_tabs = pd.crosstab(df[product], df["Churn Label"])

    chi2 = chi2_contingency(cross_tabs)[0]
    n = cross_tabs.sum().sum()
    dof = min(cross_tabs.shape) - 1
    v = np.sqrt(chi2 / (n * dof))
    
    # Interpreting the effect size
    effect_size = "Small Effect"
    for i, row in sizes.iterrows():
        if dof == row["Degrees of Freedom"]:
            if v > row["Small Effect"] and v <= row["Medium Effect"]:
                effect_size = "Small Effect"
            elif v > row["Medium Effect"] and v <= row["Large Effect"]:
                effect_size = "Medium Effect"
            else:
                effect_size = "Large Effect"
            break

    output = pd.DataFrame({
        "V" : [round(v, 3)],
        "Cramer's V Degrees of Freedom" : [dof],
        "Effect Size" : [effect_size]
    })
    return output


cramers_v(products[0])

,V,Cramer's V Degrees of Freedom,Effect Size
0,0.347,1,Medium Effect


In [112]:
print(df[["Games Product", "CLTV (Predicted Thou. IDR)"]])

            Games Product  CLTV (Predicted Thou. IDR)
0                     Yes                      4210.7
1                      No                      3511.3
2                      No                      6983.6
3                      No                      6503.9
4                      No                      6942.0
...                   ...                         ...
7038  No internet service                      6897.8
7039                  Yes                      2782.0
7040                   No                      7228.0
7041                  Yes                      3630.9
7042                  Yes                      6626.1

[7043 rows x 2 columns]
